<a href="https://colab.research.google.com/github/goracijCerv/Projectos_escolares/blob/main/EdaBinParalelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
#Librerias
import numpy as np
import math
import random
from numba import cuda

El siguiente apartado establece los terminos del experimento, es importante que no se modique el apartado de filas dado que este esta complenta mente establecido por la cantidad de "pesos".

In [111]:
#Constantes
POBLACION=20
FILAS=100
MITAD=10 
GENERACIONES=20

# > Bloque con sangría
El siguiente apartado contiene las funciones normales del programa.


In [107]:
#Funcion que ayuda a generar la solucion inicial
def genSolIni():
    generadorin = np.zeros((FILAS, 2));
    for x in range(FILAS):
        generadorin[x, 0] = random.random()
        generadorin[x, 1] = x
    ##Ordenamos el generador inicial
    generadorin = generadorin[generadorin[:, 0].argsort()]
    return  generadorin[:,1];

##Funcion de Evaluacion
def Evaluar(MatPoblacion):
    columnaCalif = np.zeros((POBLACION, 1));
    MatPoblacionCalif = np.append(MatPoblacion, columnaCalif, axis=1)
    pesos = np.array([3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9,
                      3, 6, 2, 1, 5, 7, 2, 4, 1, 9])
    cont = 0;
    bin = 0
    for x in range(POBLACION):
        y = 0;
        while (y != FILAS):
            pos = MatPoblacionCalif[x, y]
            pos = int(pos)
            p = pesos[pos]
            if (p + cont) <= 10:
                cont = cont + p
                y = y + 1
            else:
                bin = bin + 1
                cont = 0;
        MatPoblacionCalif[x, FILAS] = bin + 1
        bin = 0
        cont=0;
    return  MatPoblacionCalif

#Selecion por elitimso del %50 de la poblacion
def Elitismo(MatPoblacionCalif):
    # Ordenamos a los mejores
    MatPoblacionCalifOrd = MatPoblacionCalif[MatPoblacionCalif[:, FILAS].argsort()]
    # Obtenmos los 15 mejores
    MatPoblacionElit = MatPoblacionCalifOrd[0:MITAD, 0:FILAS]
    return MatPoblacionElit

##Obtner matriz de porcentaje para nueva generacion
def NuevoPorcentaje(MatPoblacionElit):
    MatProbabilidad = np.zeros((FILAS, FILAS))
    for x in range(FILAS):
        
        #Estableciendo los contadores y la posicion a establecer
        contadores = np.zeros((FILAS, 2));
        for x2 in range(FILAS):
          contadores[x2, 0] = 0
          contadores[x2, 1] = x2

        for y in range(MITAD):
            for x2 in range(FILAS):
              if(MatPoblacionElit[y, x]== contadores[x2,1]):
                contadores[x2,0]=contadores[x2,0]+1

        for x2 in range(FILAS):
          MatProbabilidad[x2,x] =contadores[x2,0]/MITAD

    return MatProbabilidad

#Obtenemos la nueva Poblacion)
def NuevaPoblacion(MatProbabilidad):
    PoblacionOrdenacion = np.zeros((POBLACION, FILAS))
    PoblacinN = np.zeros((POBLACION, FILAS))

    #Se obtienen los nuevos valores random
    for x in range(POBLACION):
        for y in range(FILAS):
            PoblacinN[x, y] = random.random()

    #Se establece los pesos
    for x in range(FILAS):
        prob = MatProbabilidad[:, x]
        probPos = prob.argsort()
        probOrd = prob[prob.argsort()]
        for y in range(POBLACION):
            k = 0;
            while (k != FILAS):
                if (probOrd[k] == 0):
                    k = k + 1;
                else:
                    loqueQueda = probOrd[k:]
                    loqueQuedaPos = probPos[k:]
                    numerosAv = np.zeros(len(loqueQueda))
                    sum = 0;
                    ##Se obtubo la suma de las probabilidades
                    for l in range(len(loqueQueda)):
                        if l == 0:
                            numerosAv[l] = loqueQueda[l];
                            sum = sum + loqueQueda[l]
                        else:
                            sum = sum + loqueQueda[l]
                            numerosAv[l] = sum;
                    ##Se establecera los rangos
                    for l in range(len(numerosAv)):
                        if l == 0:
                            if PoblacinN[y, x] <= numerosAv[l]:
                                PoblacionOrdenacion[y, x] = loqueQuedaPos[l];
                        elif l == len(numerosAv) - 1:
                            if PoblacinN[y, x] > numerosAv[l - 1] and PoblacinN[y, x] <= 1:
                                PoblacionOrdenacion[y, x] = loqueQuedaPos[l]
                        else:
                            if PoblacinN[y, x] > numerosAv[l - 1] and PoblacinN[y, x] <= numerosAv[l]:
                                PoblacionOrdenacion[y, x] = loqueQuedaPos[l]
                    k = FILAS;
    return  PoblacionOrdenacion

#Se eliman los pesos que se repitan más de una vez sabemos esto dado que estamos trabajando con las posiciones del peso
#asi que si se repite más de una vez la posicion se quitaran dichos elementos y se pondran otras posiciones que no aparescan
#dado que el problema establece que aparescan todos
def EliminarRep(PoblacionOrdenacion,MatProbabilidad):
    for x in range(POBLACION):
        sol = PoblacionOrdenacion[x, :]
        #Estableciendo los contadores y la posicion a establecer
        contadores = np.zeros((FILAS, 2));
        for x2 in range(FILAS):
          contadores[x2, 0] = 0
          contadores[x2, 1] = x2

        posRepetidos = []
        for y in range(FILAS):
            posicion = y + 1
            while (posicion != FILAS):
                if sol[y] == sol[posicion]:
                    posRepetidos.append(posicion)

                posicion = posicion + 1

        #Elimna las posiciones rapetidas
        for pos in posRepetidos:
            if posRepetidos.count(pos) > 1:
                posRepetidos.remove(pos)


        for k in range(FILAS):
            for k2 in range(FILAS):
              if (sol[k] == contadores[k2,1]):
                contadores[k2,0]=contadores[k2,0]+1;


        Faltantes = []
        for k in range(FILAS):
          if(contadores[k,0] == 0):
            Faltantes.append(contadores[k,1]);

        for j in range(len(posRepetidos)):
          probR=random.random()
          apuntPos=posRepetidos[j]
          probaFaltantes=PrbabilidadFaltantes(Faltantes,apuntPos,MatProbabilidad)
          print(probaFaltantes)
          lol=False
          while(lol!=True):
              for l in range(len(probaFaltantes)):
                  if l == 0:
                      if probR <= probaFaltantes[l]:
                          PoblacionOrdenacion[x,apuntPos] = Faltantes[l];
                          Faltantes.pop(l)
                  elif l == len(probaFaltantes) - 1:
                      if probR > probaFaltantes[l - 1] and probR <= 1:
                          PoblacionOrdenacion[x,apuntPos] = Faltantes[l]
                          Faltantes.pop(l)
                  else:
                      if probR > probaFaltantes[l - 1] and probR <= probaFaltantes[l]:
                          PoblacionOrdenacion[x,apuntPos] = Faltantes[l]
                          Faltantes.pop(l)

              lol=True
    return PoblacionOrdenacion

def Orden(MatPoblacionCalif):
    # Ordenamos a los mejores
    MatPoblacionCalifOrd = MatPoblacionCalif[MatPoblacionCalif[:, FILAS].argsort()]
    return MatPoblacionCalifOrd

Estas siguientes funciones son las unidades de control del paralelismo y las funciones paralelas

In [108]:
##Se obtiene la probabilidad de los que faltan
def PrbabilidadFaltantes(Faltantes, columna, matProbabilidades):
    vectroresFaltantes=np.array(Faltantes)
    col=columna
    matproblo=matProbabilidades
    listaDeProbailidadRep = np.zeros(len(vectroresFaltantes))
    probabilidadMatFaltantes = np.zeros(len(vectroresFaltantes))
    #probabilidadRe=np.zeros(len(vectroresFaltantes))
    #print( vectroresFaltantes)
    if vectroresFaltantes.shape[0]==0:
      return np.array([])
    else:
      #Bloques
      threadsperblock = (16, 16)
      blockspergrid_x = int(np.ceil( probabilidadMatFaltantes.shape[0]/ threadsperblock[0]))
      blockspergrid_y = int(np.ceil(probabilidadMatFaltantes.shape[0] / threadsperblock[1]))
      blockspergrid = (blockspergrid_x, blockspergrid_y)
      #Primera funcion en paralelo
      primera_parte[blockspergrid, threadsperblock](vectroresFaltantes,col,matproblo,probabilidadMatFaltantes)


      for x in range(len(listaDeProbailidadRep)):
          listaDeProbailidadRep[x] = probabilidadMatFaltantes[x]/np.sum(probabilidadMatFaltantes)
      #se ordena
      listaDeProbailidadRepOrdenada= np.sort(listaDeProbailidadRep)

      #En caso que los elementos sean un nan
      contNan=0

      for k in range(len(listaDeProbailidadRepOrdenada)):
          if math.isnan(listaDeProbailidadRepOrdenada[k]):
              contNan=contNan+1

      if contNan >= 1:
          listaDeProbailidadNan = np.zeros(len(listaDeProbailidadRepOrdenada))
          probabilidadesRep = 0;
          for r in range(len(listaDeProbailidadNan)):
              if r == 0:
                  listaDeProbailidadNan[r] = 1 / len(listaDeProbailidadNan);
                  probabilidadesRep = probabilidadesRep + 1 / len(listaDeProbailidadNan)
              else:
                  probabilidadesRep = probabilidadesRep + 1 / len(listaDeProbailidadNan)
                  listaDeProbailidadNan[r] = probabilidadesRep;
          return listaDeProbailidadNan

      return listaDeProbailidadRepOrdenada;

@cuda.jit
def primera_parte(VectoresFaltantes, Coulumna, MatrisProbabilidades, probabilidadMatFaltanteess):
   tid = cuda.grid(1)
   size=len(probabilidadMatFaltanteess)
   tem=0
   if tid< size:
     for k in range(MatrisProbabilidades.shape[0]):
       for k2 in range(MatrisProbabilidades.shape[1]):
         if k == Coulumna:
           for k3 in range(VectoresFaltantes.shape[0]):
             if k2 == VectoresFaltantes[k3]:
                tem=tem+MatrisProbabilidades[k2,k]
                probabilidadMatFaltanteess[tid]=MatrisProbabilidades[k2,k]
             
    


In [114]:
##Control del PRogrma
gen=0
MatPoblacion=np.zeros((POBLACION,FILAS))
#generamos las soluciones iniciales
for x in range(POBLACION):
    MatPoblacion[x]=genSolIni()
MatPoblacion = Evaluar(MatPoblacion) #evaluamos las soluciones
print("la generacion "+ str(gen+1))
print(Orden(MatPoblacion))
MatPoblacion = Elitismo(MatPoblacion) #obtenmos lo elite
MatProb= NuevoPorcentaje(MatPoblacion) #obtenmos el nuevo porcentaje para la nueva poblacion
MatPoblacion= NuevaPoblacion(MatProb) ##SE genera la nueva poblacion
MatPoblacion= EliminarRep(MatPoblacion,MatProb) #Se eliminan las posiciones repetidas
MatPoblacion = Evaluar(MatPoblacion) # Se evalua la nueva poblacion
gen = gen +1
#print("la generacion "+ str(gen+1))
#print(Orden(MatPoblacion))
while (gen != GENERACIONES):
    MatPoblacion = Elitismo(MatPoblacion)  # obtenmos lo elite
    MatProb = NuevoPorcentaje(MatPoblacion)  # obtenmos el nuevo porcentaje para la nueva poblacion
    MatPoblacion = NuevaPoblacion(MatProb)  ##SE genera la nueva poblacion
    MatPoblacion = EliminarRep(MatPoblacion,MatProb)  # Se eliminan las posiciones repetidas
    MatPoblacion = Evaluar(MatPoblacion)  # Se evalua la nueva poblacion
    gen = gen + 1
#La ultima
print("la generacion " + str(gen + 1))
print(Orden(MatPoblacion))

la generacion 1
[[41.  1. 40. ... 32. 97. 48.]
 [81. 96.  6. ... 83. 19. 49.]
 [44. 53. 87. ...  5. 66. 50.]
 ...
 [69. 41. 88. ... 75. 44. 52.]
 [66. 33. 24. ... 10. 12. 52.]
 [27. 45.  8. ... 31. 23. 53.]]
[0.02857143 0.05714286 0.08571429 0.11428571 0.14285714 0.17142857
 0.2        0.22857143 0.25714286 0.28571429 0.31428571 0.34285714
 0.37142857 0.4        0.42857143 0.45714286 0.48571429 0.51428571
 0.54285714 0.57142857 0.6        0.62857143 0.65714286 0.68571429
 0.71428571 0.74285714 0.77142857 0.8        0.82857143 0.85714286
 0.88571429 0.91428571 0.94285714 0.97142857 1.        ]
[0.02941176 0.05882353 0.08823529 0.11764706 0.14705882 0.17647059
 0.20588235 0.23529412 0.26470588 0.29411765 0.32352941 0.35294118
 0.38235294 0.41176471 0.44117647 0.47058824 0.5        0.52941176
 0.55882353 0.58823529 0.61764706 0.64705882 0.67647059 0.70588235
 0.73529412 0.76470588 0.79411765 0.82352941 0.85294118 0.88235294
 0.91176471 0.94117647 0.97058824 1.        ]
[0.03030303 0.06060

<ipython-input-108-1e06377c81eb>:23: RuntimeWarning: invalid value encountered in double_scalars
  listaDeProbailidadRep[x] = probabilidadMatFaltantes[x]/np.sum(probabilidadMatFaltantes)


Se han truncado las últimas 5000 líneas del flujo de salida.
 0.41176471 0.47058824 0.52941176 0.58823529 0.64705882 0.70588235
 0.76470588 0.82352941 0.88235294 0.94117647 1.        ]
[0.0625 0.125  0.1875 0.25   0.3125 0.375  0.4375 0.5    0.5625 0.625
 0.6875 0.75   0.8125 0.875  0.9375 1.    ]
[0.06666667 0.13333333 0.2        0.26666667 0.33333333 0.4
 0.46666667 0.53333333 0.6        0.66666667 0.73333333 0.8
 0.86666667 0.93333333 1.        ]
[0.07142857 0.14285714 0.21428571 0.28571429 0.35714286 0.42857143
 0.5        0.57142857 0.64285714 0.71428571 0.78571429 0.85714286
 0.92857143 1.        ]
[0.07692308 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308
 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308 0.07692308
 0.07692308]
[0.08333333 0.16666667 0.25       0.33333333 0.41666667 0.5
 0.58333333 0.66666667 0.75       0.83333333 0.91666667 1.        ]
[0.09090909 0.18181818 0.27272727 0.36363636 0.45454545 0.54545455
 0.63636364 0.72727273 0.81818182 0.90909091 1